In [1]:
import getpass
import os
from langchain_groq import ChatGroq

In [2]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

 ········


In [3]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableSequence, RunnableMap, RunnableParallel
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.messages import SystemMessage, trim_messages
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_chroma import Chroma

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [4]:
LLAMA_8B = "llama3-8b-8192"
LLAMA_70B = "llama3-70b-8192"
GEMMA2_9B = "gemma2-9b-it"

model = ChatGroq(model=LLAMA_8B)

In [5]:
llm = model

In [10]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FCFFE21E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FCFFD83B90>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

In [7]:
from langgraph.graph import START, StateGraph, END
from typing_extensions import List, TypedDict

In [30]:
from pydantic import BaseModel
from collections.abc import Callable
from langchain_core.documents import Document
from typing import Optional, Any, Iterable, Iterator, Generator

 - `class collections.abc.Iterator`

$ \hspace{10mm} $ ABC for classes that provide the `__iter__()` and `__next__()` methods.


 - `class collections.abc.Iterable`

$ \hspace{10mm} $ ABC for classes that provide the `__iter__()` method.

 - `class collections.abc.Generator`

$ \hspace{10mm} $ ABC for generator classes that implement the protocol defined in PEP 342 that extends iterators with the `send()`, `throw()` and `close()` methods.
.

### Basic calculator agent

In [22]:
# @tool
def add(a: float, b: float) -> float:
    """Adds two numbers."""
    return a + b

# @tool
def subtract(a: float, b: float) -> float:
    """Subtracts two numbers."""
    return a - b

# @tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers."""
    return a * b

# @tool
def divide(a: float, b: float) -> float:
    """Divides two numbers."""
    if b == 0:
        return "Cannot divide by zero."
    return a / b


tools = [add, subtract, multiply, divide]

In [46]:
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.runnables.graph import Graph
from langchain_core.runnables.base import Runnable


In [47]:
issubclass(Graph, Runnable)

False

In [67]:
class State(TypedDict):
    question: str
    answer: str
    context: Optional[List[Document]]


class Agent(BaseModel):
    model: BaseChatModel
    tools: List[Callable]
    context: Optional[List[Document]]
    workflow: Any = None


    def _create_agent_workflow(self):
        def bind(state: State):
            llm_with_tools = self.model.bind_tools(tools)
            response = llm_with_tools.invoke(state["question"])
            return {"answer": [response]}

        toolnode = ToolNode(tools)
        tools_cache = {tool.__name__: tool for tool in tools}

        def call_tool(state: State):
            last_message = state["answer"][-1]
            tool_call = last_message.tool_calls[0]
            tool_name = tool_call["name"]
            arguments = tool_call["args"]
        
            if tool_name not in tools_cache:
                output = f"Tool `{tool_name}` is not defined"

            else:
                output = tools_cache[tool_name](**arguments)
        
            return {
                "question": state["question"],
                "answer": str(output),
            }

        graph_builder = StateGraph(State)

        graph_builder.add_node(bind)
        graph_builder.add_node(toolnode)
        graph_builder.add_node(call_tool)
        
        graph_builder.add_edge(START, "bind")
        graph_builder.add_edge("bind", "call_tool")
        self.workflow = graph_builder.compile()


    def run(self, input):
        if not self.workflow:
            print("Initializing workflow")
            self._create_agent_workflow()

        response = self.workflow.invoke(input)
        return response

In [68]:
calculator = Agent(model=model, tools=tools, context=[])

In [69]:
calculator.run({"question": "What if you have 3 and then I take one from you?"})

Initializing workflow


{'question': 'What if you have 3 and then I take one from you?', 'answer': '2'}

### Basic calculator agent

In [15]:
class State(TypedDict):
    question: str
    answer: str

In [19]:
def calculate(state: State):
    llm_with_tools = llm.bind_tools(tool_list)
    response = llm_with_tools.invoke(state["question"])
    return {"answer": [response]}


tools = ToolNode(tool_list)


def call_tool(state: State):
    last_message = state["answer"][-1]
    tool_call = last_message.tool_calls[0]
    function_name = tool_call["name"]
    arguments = tool_call["args"]

    if function_name == "add":
        output = add(**arguments)
    elif function_name == "subtract":
         output = subtract(**arguments)
    elif function_name == "multiply":
        output = multiply(**arguments)
    elif function_name == "divide":
         output = divide(**arguments)
    else:
        output = f"Tool `{function_name}` is not defined"

    return {
        "question": state["question"],
        "answer": str(output),
    }

In [64]:
graph_builder = StateGraph(State)

graph_builder.add_node(calculate)
graph_builder.add_node(tools)
graph_builder.add_node(call_tool)

graph_builder.add_edge(START, "calculate")
graph_builder.add_edge("calculate", "call_tool")
graph = graph_builder.compile()

In [65]:
"What if you have 2 and then you have one more?"

'What if you have 2 and then you have one more?'

In [66]:
response = graph.invoke({"question": "What if you have 3 and then I take one from you?"})
print(response["answer"])

2


In [67]:
response

{'question': 'What if you have 3 and then I take one from you?', 'answer': '2'}

### Basic calculator agent with system prompt

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful calculator. You have the following functions available to you: {functions} \n. When using a function, reply in the following format. \n```tool_code\n{{ function_name: <function name>, arguments: {{ arg1: <value>, arg2: <value> }} }} \n```"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [10]:
class CalculatorState(TypedDict):
    messages: List[AIMessage | HumanMessage]
    tool_output: str = None
    output: str = None

In [16]:
def call_llm(state: CalculatorState):
    """Call the LLM to decide next step."""
    runnable = (prompt | llm.bind_tools(tool_list))
    runnable.invoke({"messages": state["messages"]})
    return {"messages": state["messages"] + [response]}


def check_tool_call(state: CalculatorState):
    """Check if llm is asking to use tool or not."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "call_tool"
    return "user_response"


def call_tool(state: CalculatorState):
    """If llm returned a tool call, call the tool"""
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = tool_call.function.arguments

    if function_name == "add":
        output = add(**arguments)
    elif function_name == "subtract":
         output = subtract(**arguments)
    elif function_name == "multiply":
        output = multiply(**arguments)
    elif function_name == "divide":
         output = divide(**arguments)
    else:
        output = f"Tool `{function_name}` is not defined"

    return {
       "messages": state.messages + [AIMessage(content="", tool_call_id=tool_call.id, tool_call_result=str(output))],
        "tool_output": output
    }


def user_response(state: CalculatorState):
    """Return last llm output to user."""
    last_message = state.messages[-1]
    return {"messages": state.messages, "output": last_message.content}


In [17]:
workflow = StateGraph(CalculatorState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("check_tool_call", check_tool_call)
workflow.add_node("call_tool", call_tool)
workflow.add_node("user_response", user_response)

workflow.set_entry_point("call_llm")

workflow.add_conditional_edges(
    "call_llm",
    check_tool_call,
    {
        "call_tool": "call_tool",
        "user_response": "user_response"
    },
)

workflow.add_edge("call_tool", "call_llm")
workflow.add_edge("user_response", END)

app = workflow.compile()

In [18]:
input_question = "What if you have 3 and then I take one from you?"

In [19]:
inputs = {"messages": [HumanMessage(content=input_question)]}

app.invoke(inputs)

KeyError: "Input to ChatPromptTemplate is missing variables {'functions'}.  Expected: ['functions', 'messages'] Received: ['messages']\nNote: if you intended {functions} to be part of the string and not a variable, please escape it with double curly braces like: '{{functions}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

In [ ]:

tools = [
    Tool(
        name="add",
        func=add,
        description="Adds two numbers.",
    ),
    Tool(
        name="subtract",
        func=subtract,
        description="Subtracts two numbers.",
    ),
    Tool(
        name="multiply",
        func=multiply,
        description="Multiplies two numbers.",
    ),
    Tool(
        name="divide",
        func=divide,
        description="Divides two numbers.",
    ),
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful calculator. You have the following functions available to you: {functions} \n. When using a function, reply in the following format. \n```tool_code\n{{ function_name: <function name>, arguments: {{ arg1: <value>, arg2: <value> }} }} \n```"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def agent(messages, agent_scratchpad):
    formatted_messages = prompt.format_messages(
        messages=messages,
        agent_scratchpad=format_to_openai_function_messages(agent_scratchpad),
    )
    response = llm.invoke(formatted_messages)
    return response

output_parser = OpenAIFunctionsAgentOutputParser()

tool_executor = ToolExecutor(tools)

class AgentState:
    messages: List
    agent_scratchpad: List
    tool_output: dict


def call_tool(state):
    """Calls the tool and updates intermediate steps"""
    response = output_parser.parse(state.intermediate_steps.pop())
    
    if response.tool not in ["add", "subtract", "multiply", "divide"]:
        return {"messages": state.messages, "agent_scratchpad":state.agent_scratchpad, "tool_output": None, "intermediate_steps":state.intermediate_steps}
    
    tool_output = tool_executor.invoke(
        {"tool_call": response.tool, "tool_input": response.tool_input}
    )
    
    state.intermediate_steps.append(AIMessage(content=str(tool_output),tool_calls=[])) # Add output as agent message
    return {"messages": state.messages, "agent_scratchpad":state.agent_scratchpad, "tool_output": tool_output, "intermediate_steps":state.intermediate_steps}
    
def end(state):
    return {"messages": state.messages, "agent_scratchpad": state.agent_scratchpad, "tool_output": None, "intermediate_steps": state.intermediate_steps}

workflow = StateGraph(AgentState)
workflow.add_node("agent", agent)
workflow.add_node("call_tool", call_tool)
workflow.add_node("end", end)

workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
      "call_tool": "call_tool",
      "end": "end"
     }
)
workflow.add_edge("call_tool","agent")

app = workflow.compile()

# 8. Run the Agent
inputs = {"messages": [HumanMessage(content="What is 2 + 2 and then multiply by 10?")] , "agent_scratchpad": [], "intermediate_steps": [], "tool_output": None}
result = app.invoke(inputs)

print("Result: ")
if result["tool_output"] is not None:
    print(result["tool_output"])
else:
    print(result["messages"][-1].content)

In [24]:
tools_dict = {tool.__name__: tool for tool in tools}

In [26]:
tools_dict["add"](1, 3)

4